In [ ]:
# =====================================================================
# FIVE-CARD DRAW — Coach en vivo (ES) 1 ronda + Recomendador de Apuestas
# - Descarta (UNA ronda), roba y FIN (mesa real).
# - NUNCA rompe jugada hecha (trío/doble pareja/pareja); Escalera+ = plantarse.
# - Integra tu best_model.pkl si está (para EV adicional).
# - Recomendación de APUESTAS por ronda: Pasar / Apostar / Igualar / Subir / Retirarse
# =====================================================================

import os, random
from itertools import combinations
from collections import Counter
from typing import Optional, List, Tuple, Dict
import numpy as np

try:
    import joblib
except Exception:
    joblib = None

random.seed(42)

# -------------------------
# Configuración general
# -------------------------
PESO_GANAR = 0.7          # peso de P(ganar) en la decisión de DESCARTE
PESO_MODELO = 0.3         # peso del EV de tu modelo (si está cargado)

UMBRAL_FUERTE_CAT = 4     # 4 = Escalera o mejor -> NO descartar (stand-pat)
RIVALES_DEF = 1           # siempre 1 rival (heads-up)
SIMS_DEF = 4000           # simulaciones por acción (descartes)

GUARDA_ALTAS = True       # evita tirar ≥2 Broadways (T,J,Q,K,A) con ganancia marginal
MEJORA_MIN_GANAR = 0.02   # ganancia mínima en P(ganar) para permitir tirar ≥2 altas

# Parámetros para RECOMENDACIÓN DE APUESTAS (puedes cambiarlos)
POT_DEF = 10.0            # tamaño del bote actual
TO_CALL_DEF = 2.0         # cantidad a pagar si te apuestan (igualar)
BET_SIZE_DEF = 2.0        # tamaño de tu apuesta si decides apostar (informativo)
RAISE_SIZE_DEF = 2.0      # incremento típico si decides subir (informativo)

# Umbrales "humanos" para apostar/subir según P(ganar)
TH_STRONG = 0.70          # si P(ganar) >= 70% -> apostar/subir
TH_MEDIUM = 0.50          # si 50%-70% -> apuesta pequeña o check-call

# -------------------------
# Cargar TU MODELO (si está disponible)
# -------------------------
RUTAS_MODELO = ["/mnt/data/best_model.pkl", "/content/best_model.pkl"]
best_model, best_model_path = None, None
if joblib is not None:
    for ruta in RUTAS_MODELO:
        if os.path.exists(ruta):
            try:
                obj = joblib.load(ruta)
                best_model = obj["model"] if isinstance(obj, dict) and "model" in obj else obj
                best_model_path = ruta
                print(f"✅ Modelo cargado: {ruta}")
                break
            except Exception as e:
                print(f"⚠️ No pude cargar el modelo desde {ruta}: {e}")
else:
    print("ℹ️ joblib no disponible: se usará solo P(ganar) para decisiones.")

# -------------------------
# Utilidades de cartas (sin símbolos, formato 'AH', '2S', etc.)
# -------------------------
RANGO_CHAR_A_NUM = {"A":14,"K":13,"Q":12,"J":11,"T":10,
                    "9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2}
PALOS = ["H","S","D","C"]  # H=corazones, S=picas, D=diamantes, C=tréboles
MAZO_COMPLETO = [r+s for r in RANGO_CHAR_A_NUM.keys() for s in PALOS]

CLASES_ES = {
    0:"Carta alta", 1:"Pareja", 2:"Doble pareja", 3:"Trío",
    4:"Escalera", 5:"Color", 6:"Full", 7:"Póker",
    8:"Escalera de color", 9:"Escalera real"
}
EN_A_ES = {
    "Nothing":"Carta alta", "High card":"Carta alta",
    "One pair":"Pareja", "Two pairs":"Doble pareja",
    "Three of a kind":"Trío", "Trips":"Trío",
    "Straight":"Escalera", "Flush":"Color",
    "Full house":"Full", "Four of a kind":"Póker",
    "Straight flush":"Escalera de color", "Royal flush":"Escalera real",
    "Nada":"Carta alta"
}

def parse_hand_str(s: str):
    partes = s.replace(",", " ").split()
    assert len(partes) == 5, "Debes escribir 5 cartas (ej.: 'AH KH QH 4S 3S')"
    mano, vistas = [], set()
    for p in partes:
        p = p.strip().upper()
        assert len(p) in (2,3), f"Carta inválida: {p}"
        r, su = (p[:-1], p[-1])
        if r == "10": r = "T"
        assert r in RANGO_CHAR_A_NUM, f"Rango inválido: {r}"
        assert su in PALOS, f"Palo inválido: {su} (usa H,S,D,C)"
        carta = (r, su)
        assert carta not in vistas, f"Carta repetida: {p}"
        vistas.add(carta); mano.append(carta)
    return mano

def parse_draw_str(s: str, k: int):
    partes = s.replace(",", " ").split()
    assert len(partes) == k, f"Debes escribir exactamente {k} carta(s) robada(s)."
    out, vistas = [], set()
    for p in partes:
        p = p.strip().upper()
        assert len(p) in (2,3), f"Carta inválida: {p}"
        r, su = (p[:-1], p[-1])
        if r == "10": r = "T"
        assert r in RANGO_CHAR_A_NUM, f"Rango inválido: {r}"
        assert su in PALOS, f"Palo inválido: {su} (usa H,S,D,C)"
        carta = (r, su)
        assert carta not in vistas, f"Carta repetida en el robo: {p}"
        vistas.add(carta); out.append(carta)
    return out

def a_str(mano: List[Tuple[str,str]]) -> str:
    return ", ".join([f"{r}{s}" for r,s in mano])

def a_str_list(mano: List[Tuple[str,str]]) -> List[str]:
    return [r+s for r,s in mano]

def es_broadway(r_char: str) -> bool:
    return RANGO_CHAR_A_NUM[r_char] >= 10  # T..A

# -------------------------
# Evaluación por REGLAS (comparación + nombre ES)
# -------------------------
def _rank_para_comparar(cartas_str: List[str]):
    """ Devuelve (cat, detalle...) para ordenar manos:
        8 Escalera de color · 7 Póker · 6 Full · 5 Color · 4 Escalera
        3 Trío · 2 Doble pareja · 1 Pareja · 0 Carta alta
    """
    rangos = sorted([RANGO_CHAR_A_NUM[c[0]] for c in cartas_str], reverse=True)
    palos = [c[1] for c in cartas_str]
    rc = Counter(rangos)
    counts = sorted(rc.values(), reverse=True)
    by_cnt_rank = sorted(((cnt, r) for r, cnt in rc.items()),
                         key=lambda x: (x[0], x[1]), reverse=True)
    es_color = len(set(palos)) == 1

    def es_consecutiva(rs):
        return all(rs[i] - 1 == rs[i+1] for i in range(len(rs)-1))

    uniq = sorted(set(rangos), reverse=True)
    es_escalera = False; tope_esc = None
    if len(uniq) == 5 and es_consecutiva(uniq):
        es_escalera = True; tope_esc = uniq[0]
    if set(rangos) == {14,5,4,3,2}:  # A-2-3-4-5
        es_escalera = True; tope_esc = 5

    if es_escalera and es_color: return (8, (tope_esc or 14,))
    if counts[0] == 4:
        poker = by_cnt_rank[0][1]; kicker = max([r for r in rangos if r != poker]); return (7,(poker,kicker))
    if counts[0] == 3 and counts[1] == 2:
        trio = by_cnt_rank[0][1]; pareja = by_cnt_rank[1][1]; return (6,(trio,pareja))
    if es_color: return (5, tuple(rangos))
    if es_escalera: return (4, (tope_esc,))
    if counts[0] == 3:
        trio = by_cnt_rank[0][1]; kick = sorted([r for r in rangos if r != trio], reverse=True); return (3,(trio,)+tuple(kick))
    if counts[0] == 2 and counts[1] == 2:
        pares = sorted([r for r,c in rc.items() if c==2], reverse=True); kicker = max([r for r,c in rc.items() if c==1]); return (2,(pares[0],pares[1],kicker))
    if counts[0] == 2:
        pareja = by_cnt_rank[0][1]; kick = sorted([r for r in rangos if r != pareja], reverse=True); return (1,(pareja,)+tuple(kick))
    return (0, tuple(rangos))

def nombre_reglas_es(mano: List[Tuple[str,str]]) -> str:
    h = a_str_list(mano)
    cat, _ = _rank_para_comparar(h)
    if cat == 8:
        rangos = sorted([RANGO_CHAR_A_NUM[c[0]] for c in h], reverse=True)
        if set(rangos) == {14,13,12,11,10}:
            return "Escalera real"
        return "Escalera de color"
    return CLASES_ES.get(cat, "Carta alta")

def cat_reglas(mano: List[Tuple[str,str]]) -> int:
    return _rank_para_comparar(a_str_list(mano))[0]

def es_fuerte_por_reglas(mano: List[Tuple[str,str]]) -> bool:
    return cat_reglas(mano) >= UMBRAL_FUERTE_CAT

# -------------------------
# Probabilidad de ganar (Monte Carlo) — UNA ronda de descarte
# -------------------------
def prob_ganar_accion(mano: List[Tuple[str,str]], idx_descartar: Tuple[int,...],
                      rivales: int = RIVALES_DEF, simulaciones: int = SIMS_DEF, rng=None) -> float:
    rng = rng or random.Random(202)
    m_str = a_str_list(mano)
    keep = [c for i,c in enumerate(m_str) if i not in idx_descartar]
    k = len(idx_descartar)
    wins = 0.0
    for _ in range(simulaciones):
        mazo = [c for c in MAZO_COMPLETO if c not in keep]
        rng.shuffle(mazo)
        robo = mazo[:k]
        mi_final = keep + robo
        pool = mazo[k:]
        rivales_manos = [pool[5*i:5*(i+1)] for i in range(rivales)]
        mi_rank = _rank_para_comparar(mi_final)
        opp_ranks = [_rank_para_comparar(h) for h in rivales_manos]
        mejor = max([mi_rank] + opp_ranks)
        n_mejor = sum(1 for r in [mi_rank]+opp_ranks if r == mejor)
        if mi_rank == mejor:
            wins += 1.0/n_mejor
    return wins / simulaciones

# -------------------------
# EV de TU MODELO (opcional)
# -------------------------
RANGO_CHAR_A_NUM_M = {"A":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,
                      "8":8,"9":9,"T":10,"J":11,"Q":12,"K":13}
PALO_CHAR_A_NUM_M  = {"H":1,"S":2,"D":3,"C":4}

def _tuplas_modelo_desde_str(cartas_str: List[str]):
    return [(RANGO_CHAR_A_NUM_M[c[0]], PALO_CHAR_A_NUM_M[c[1]]) for c in cartas_str]

def _longitud_esc_m(rangos_1a13: List[int]) -> int:
    s = sorted(set(rangos_1a13))
    if not s: return 0
    best = cur = 1
    for i in range(1,len(s)):
        if s[i] == s[i-1] + 1:
            cur += 1; best = max(best, cur)
        else:
            cur = 1
    return min(best, 5)

def _feats_modelo(tuples_) -> Dict[str, float]:
    rangos = [int(r) for r,_ in tuples_]
    palos  = [int(s) for _,s in tuples_]
    rc = Counter(rangos); sc = Counter(palos)
    feats = {
        "max_equal_ranks": max(rc.values()),
        "num_pairs": sum(1 for v in rc.values() if v == 2),
        "has_three_kind": int(any(v == 3 for v in rc.values())),
        "has_four_kind": int(any(v == 4 for v in rc.values())),
        "max_equal_suits": max(sc.values()),
        "num_suits_distinct": len(sc),
        "longest_sequence": _longitud_esc_m(rangos_1a13=rangos),
        "has_royal_structure": int(set(rangos) == {1,10,11,12,13}),
        "num_ranks_distinct": len(rc),
        "is_flush": int(max(sc.values()) == 5),
        "is_straight": int(_longitud_esc_m(rangos) == 5 and max(sc.values()) != 5),
        "is_straight_or_better": int(_longitud_esc_m(rangos) == 5),
        "is_full_house": int(any(v==3 for v in rc.values()) and any(v==2 for v in rc.values())),
        "is_two_pairs": int(sum(1 for v in rc.values() if v==2) == 2),
        "is_one_pair": int(sum(1 for v in rc.values() if v==2) == 1 and not any(v==3 for v in rc.values()) and not any(v==4 for v in rc.values())),
        "is_three_kind": int(any(v==3 for v in rc.values())),
        "is_four_kind": int(any(v==4 for v in rc.values())),
        "is_straight_flush": int(max(sc.values())==5 and _longitud_esc_m(rangos)==5 and not set(rangos)=={1,10,11,12,13}),
        "is_royal_flush": int(max(sc.values())==5 and set(rangos)=={1,10,11,12,13}),
        "sum_ranks": sum(rangos),
        "max_rank": max(rangos),
        "min_rank": min(rangos),
    }
    return feats

def _features_requeridas(modelo) -> Optional[list]:
    if modelo is None: return None
    if hasattr(modelo, "feature_names_in_"):
        try: return list(modelo.feature_names_in_)
        except Exception: pass
    steps = getattr(modelo, "named_steps", None)
    if steps:
        last = steps[list(steps.keys())[-1]]
        if hasattr(last, "feature_names_in_"):
            return list(last.feature_names_in_)
    return None

def _label_a_valor(lbl):
    if isinstance(lbl, (int, np.integer)): return int(lbl)
    m = {"Carta alta":0,"Pareja":1,"Doble pareja":2,"Trío":3,"Escalera":4,"Color":5,"Full":6,"Póker":7,"Escalera de color":8,"Escalera real":9,
         "Nothing":0,"High card":0,"One pair":1,"Two pairs":2,"Three of a kind":3,
         "Straight":4,"Flush":5,"Full house":6,"Four of a kind":7,"Straight flush":8,"Royal flush":9,"Nada":0}
    return m.get(str(lbl), 0)

def _label_a_es(lbl):
    if isinstance(lbl, (int, np.integer)): return CLASES_ES.get(int(lbl), "Carta alta")
    s = str(lbl)
    if s in CLASES_ES.values(): return s
    return EN_A_ES.get(s, "Carta alta")

def nombre_modelo_es(mano: List[Tuple[str,str]]) -> str:
    if best_model is None: return "—"
    try:
        req = _features_requeridas(best_model)
        t = _tuplas_modelo_desde_str(a_str_list(mano))
        feats = _feats_modelo(t)
        if req is None:
            x = np.array([feats[k] for k in sorted(feats.keys())], dtype=float).reshape(1,-1)
        else:
            if any(c not in feats for c in req): return "—"
            x = np.array([feats[c] for c in req], dtype=float).reshape(1,-1)
        pred = best_model.predict(x)[0]
        return _label_a_es(pred)
    except Exception:
        return "—"

def ev_modelo_para_accion(mano: List[Tuple[str,str]], idx_descartar: Tuple[int,...],
                          modelo=best_model, simulaciones: int = max(1000, SIMS_DEF//2), rng=None) -> Optional[float]:
    if modelo is None: return None
    rng = rng or random.Random(303)
    keep = [c for i,c in enumerate(a_str_list(mano)) if i not in idx_descartar]
    k = len(idx_descartar); req = _features_requeridas(modelo)
    vals = []
    for _ in range(simulaciones):
        mazo = [c for c in MAZO_COMPLETO if c not in keep]
        rng.shuffle(mazo); robo = mazo[:k]; final = keep + robo
        t = _tuplas_modelo_desde_str(final); feats = _feats_modelo(t)
        if req is None:
            x = np.array([feats[k] for k in sorted(feats.keys())], dtype=float).reshape(1,-1)
        else:
            if any(c not in feats for c in req): return None
            x = np.array([feats[c] for c in req], dtype=float).reshape(1,-1)
        try:
            if hasattr(modelo, "predict_proba"):
                probs = modelo.predict_proba(x)[0]
                clases = getattr(modelo, "classes_", None)
                if clases is not None:
                    vals_clase = np.array([_label_a_valor(c) for c in clases], dtype=float)
                    val = float(np.dot(probs, vals_clase))
                else:
                    val = float(np.dot(probs, np.arange(len(probs), dtype=float)))
            else:
                pred = modelo.predict(x)[0]
                val = float(_label_a_valor(pred))
            vals.append(val)
        except Exception:
            return None
    return float(np.mean(vals)) if vals else None

# -------------------------
# Guardarraíl para NO romper jugadas hechas
# -------------------------
def _estructura_mano_indices(mano: List[Tuple[str,str]]):
    h = [r+s for r,s in mano]
    cat, _ = _rank_para_comparar(h)
    rank_to_idx = {}
    for i,(r,s) in enumerate(mano):
        rank_to_idx.setdefault(r, []).append(i)
    return cat, rank_to_idx

def _indices_kickers_para_descartar(mano: List[Tuple[str,str]]) -> Tuple[int, List[int]]:
    cat, groups = _estructura_mano_indices(mano)
    sizes = sorted((len(v) for v in groups.values()), reverse=True)
    if 3 in sizes and 2 not in sizes and 4 not in sizes:  # Trío
        trio_rank = next(r for r,idxs in groups.items() if len(idxs)==3)
        kickers = [i for r,idxs in groups.items() if r!=trio_rank for i in idxs]
        return cat, kickers
    if sizes[:2] == [2,2]:  # Doble pareja
        singleton_rank = next(r for r,idxs in groups.items() if len(idxs)==1)
        return cat, groups[singleton_rank]
    if sizes[0] == 2 and sizes.count(2) == 1:  # Pareja
        pair_rank = next(r for r,idxs in groups.items() if len(idxs)==2)
        kickers = [i for r,idxs in groups.items() if r!=pair_rank for i in idxs]
        return cat, kickers
    return cat, []

def acciones_posibles(mano: List[Tuple[str,str]], max_desc: int = 3) -> List[Tuple[int,...]]:
    cat = cat_reglas(mano)
    if cat >= 4:  # Escalera+
        return [()]
    _, kickers = _indices_kickers_para_descartar(mano)
    if cat == 3 and len(kickers) == 2:  # Trío
        return [tuple(sorted(kickers)), ()]
    if cat == 2 and len(kickers) == 1:  # Doble pareja
        return [tuple(kickers), ()]
    if cat == 1 and len(kickers) == 3:  # Pareja
        return [tuple(sorted(kickers)), ()]
    # Carta alta: 0..max_desc
    acts = [()]
    for k in range(1, max_desc+1):
        acts += list(combinations(range(5), k))
    return acts

def _minmax_norm(vals: List[float]) -> List[float]:
    arr = np.array(vals, dtype=float)
    vmin, vmax = np.nanmin(arr), np.nanmax(arr)
    if not np.isfinite(vmin) or not np.isfinite(vmax): return [np.nan]*len(vals)
    if vmax > vmin: return list((arr - vmin) / (vmax - vmin))
    return [0.5]*len(vals)

def elegir_mejor_accion(mano: List[Tuple[str,str]],
                        rivales: int = RIVALES_DEF,
                        simulaciones: int = SIMS_DEF,
                        modelo = best_model):
    if es_fuerte_por_reglas(mano):
        return {"action": (), "win_prob": 1.0, "score": 1.0, "model_ev": None}, [{"action": (), "win_prob": 1.0, "score": 1.0, "model_ev": None}]
    acts = acciones_posibles(mano, max_desc=3)
    filas = []
    for a in acts:
        pwin = prob_ganar_accion(mano, a, rivales=rivales, simulaciones=simulaciones)
        mev  = ev_modelo_para_accion(mano, a, modelo=modelo, simulaciones=max(1000, simulaciones//2))
        filas.append({"action": a, "win_prob": pwin, "model_ev": mev})
    wp_norm = _minmax_norm([r["win_prob"] for r in filas])
    mev_vals = [(-1 if r["model_ev"] is None else r["model_ev"]) for r in filas]
    if any(v >= 0 for v in mev_vals):
        tmp = [np.nan if v < 0 else v for v in mev_vals]
        media = float(np.nanmean(np.array(tmp, dtype=float)))
        tmp2 = [media if (v is np.nan or v!=v) else v for v in tmp]
        mev_norm = _minmax_norm(tmp2)
    else:
        mev_norm = [np.nan]*len(filas)
    for r, wpn, men in zip(filas, wp_norm, mev_norm):
        w1 = PESO_GANAR; w2 = PESO_MODELO if np.isfinite(men) else 0.0
        denom = (w1 + w2) if (w1 + w2) > 0 else 1.0
        r["score"] = float((w1*wpn + w2*men) / denom)
    base = next(row for row in filas if len(row["action"]) == 0)
    filas.sort(key=lambda d: (d["score"], -len(d["action"])), reverse=True)
    best = filas[0]
    if GUARDA_ALTAS and len(best["action"]) >= 2:
        a_tirar = [mano[i] for i in best["action"]]
        if sum(es_broadway(r) for r,_ in a_tirar) >= 2:
            gain = best["win_prob"] - base["win_prob"]
            if gain < MEJORA_MIN_GANAR: best = base
    return best, filas

# -------------------------
# Mensajes de descarte (kicker-aware)
# -------------------------
def consejo_jugador_kicker_aware(mano: List[Tuple[str,str]], best_accion: Dict) -> str:
    k = len(best_accion["action"])
    tirar_idx = set(best_accion["action"])
    tirar = [mano[i] for i in best_accion["action"]]
    guardar = [c for i,c in enumerate(mano) if i not in tirar_idx]
    nombre = nombre_reglas_es(mano)
    cat = cat_reglas(mano)
    if k == 0:
        return f"Tienes {nombre}. **Plántate** (no descartes)."
    if cat == 3 and k == 2:
        return f"Tienes Trío. Descarta las 2 sueltas: {a_str(tirar)}. Conserva {a_str(guardar)}. Roba 2."
    if cat == 2 and k == 1:
        return f"Tienes Doble pareja. Descarta la carta suelta: {a_str(tirar)}. Conserva {a_str(guardar)}. Roba 1."
    if cat == 1 and k == 3:
        return f"Tienes Pareja. Conserva la pareja y descarta 3: {a_str(tirar)}. Roba 3."
    if cat == 0 and k >= 1:
        if k == 3:
            return f"Tienes Carta alta. Descarta 3 ({a_str(tirar)}). Roba 3."
        return f"Tienes Carta alta. Descarta {k}: {a_str(tirar)}. Roba {k}."
    return f"Descarta {a_str(tirar)}. Conserva {a_str(guardar)}. Roba {k}."

# -------------------------
# Recomendador de Apuestas (una ronda)
# -------------------------
def breakeven_prob(to_call: float, pot: float) -> float:
    """ Probabilidad mínima para que igualar sea rentable: to_call / (pot + to_call) """
    if pot + to_call <= 0: return 1.0
    return to_call / (pot + to_call)

def betting_advice_text(p_win: float, pot: float = POT_DEF, to_call: float = TO_CALL_DEF) -> str:
    """
    Devuelve texto 'para humanos' con las opciones de la ronda y la mejor recomendación.
    Opciones: Pasar · Apostar · Igualar · Subir · Retirarse
    """
    be = breakeven_prob(to_call, pot)
    p = p_win
    base = f"Opciones: Pasar · Apostar · Igualar · Subir · Retirarse | P(ganar)≈{p*100:.1f}% · Breakeven igualar≈{be*100:.1f}%"
    if p >= TH_STRONG:
        return base + "\nRecomendación: **Apostar/Subir** (mano fuerte). Evita dar cartas gratis."
    if p >= max(be, TH_MEDIUM):
        return base + "\nRecomendación: **Apostar pequeño** o **Pasar** esperando **Igualar** (control de bote)."
    if p >= be:
        return base + "\nRecomendación: **Igualar** si la apuesta es razonable; si suben fuerte por encima del breakeven, **Retirarse**."
    return base + "\nRecomendación: **Pasar** y ante apuesta grande **Retirarse** (check-fold)."

# -------------------------
# Aplicar robo (ÚNICA ronda)
# -------------------------
def aplicar_robo(mano: List[Tuple[str,str]], best_accion: Dict, robadas_str: str):
    k = len(best_accion["action"])
    robadas = parse_draw_str(robadas_str, k)
    guardar = [c for i,c in enumerate(mano) if i not in best_accion["action"]]
    nueva = guardar + robadas
    if len(nueva) != len({tuple(c) for c in nueva}):
        raise AssertionError("Hay cartas duplicadas en la mano final.")
    return nueva

def pedir_si_no(msg="¿Nueva ronda? (S/N): "):
    while True:
        r = input(msg).strip().upper()
        if r in ("S","N"):
            return r
        print("Responde 'S' o 'N', por favor.")

# -------------------------
# CLI en vivo (1 ronda, con apuesta)
# -------------------------
def play_loop():
    print("== Five-Card Draw — Coach en vivo (ES) 1 ronda + Apuestas ==")
    if best_model_path:
        print(f"Modelo cargado: {best_model_path}")
    else:
        print("Sin modelo cargado: se usa SOLO P(ganar).")
    print(f"Descartes: 1 ronda | Rivales={RIVALES_DEF} | Sims/acción={SIMS_DEF} | Stand-pat desde {CLASES_ES[UMBRAL_FUERTE_CAT]}")
    print(f"Apuestas: pot={POT_DEF}, to_call={TO_CALL_DEF}\n")

    while True:
        s = input("Mano (5 cartas, ej. 'AH KH QH 4S 3S') o 'salir': ").strip()
        if s.lower() == "salir":
            print("Fin."); break

        try:
            mano = parse_hand_str(s)
        except AssertionError as e:
            print("❌", e); continue

        nombre_reg = nombre_reglas_es(mano)
        nombre_mod = nombre_modelo_es(mano)
        print(f"Tu mano: {a_str(mano)} | Reglas: {nombre_reg} | Modelo: {nombre_mod}")

        # ¿Ya es fuerte? => plantarse (no descartar) y recomendar apuesta
        if es_fuerte_por_reglas(mano):
            print("→ Mano FUERTE por reglas. **Plántate** (no descartes).")
            pwin = prob_ganar_accion(mano, (), rivales=RIVALES_DEF, simulaciones=SIMS_DEF)
            print(betting_advice_text(pwin, pot=POT_DEF, to_call=TO_CALL_DEF))
            print("Jugada finalizada.")
            if pedir_si_no() == "N": print("Fin."); break
            print(""); continue

        # Elegir mejor acción de DESCARTE
        best, _ = elegir_mejor_accion(mano, rivales=RIVALES_DEF, simulaciones=SIMS_DEF, modelo=best_model)
        print("→", consejo_jugador_kicker_aware(mano, best))

        k = len(best["action"])
        # Si recomienda plantarse (k=0), pasar a apuestas y cerrar
        if k == 0:
            pwin = prob_ganar_accion(mano, (), rivales=RIVALES_DEF, simulaciones=SIMS_DEF)
            print(betting_advice_text(pwin, pot=POT_DEF, to_call=TO_CALL_DEF))
            print("Jugada finalizada.")
            if pedir_si_no() == "N": print("Fin."); break
            print(""); continue

        # ÚNICO robo
        while True:
            rob = input(f"Indica las {k} carta(s) ROBADAS (ej. '2D 7H ...'): ").strip()
            try:
                mano_final = aplicar_robo(mano, best, rob)
                break
            except AssertionError as e:
                print("❌", e, "Vuelve a introducir las cartas.")

        # Nombre y apuestas post-draw
        print(f"Mano final: {a_str(mano_final)} | Reglas: {nombre_reglas_es(mano_final)} | Modelo: {nombre_modelo_es(mano_final)}")
        pwin_final = prob_ganar_accion(mano, best["action"], rivales=RIVALES_DEF, simulaciones=SIMS_DEF)
        print(betting_advice_text(pwin_final, pot=POT_DEF, to_call=TO_CALL_DEF))
        print("Jugada finalizada.")
        if pedir_si_no() == "N":
            print("Fin."); break
        print("")

# -------------------------
# Arranque
# -------------------------
if __name__ == "__main__":
    play_loop()


✅ Modelo cargado: /content/best_model.pkl
== Five-Card Draw — Coach en vivo (ES) 1 ronda + Apuestas ==
Modelo cargado: /content/best_model.pkl
Descartes: 1 ronda | Rivales=1 | Sims/acción=4000 | Stand-pat desde Escalera
Apuestas: pot=10.0, to_call=2.0

Mano (5 cartas, ej. 'AH KH QH 4S 3S') o 'salir': AH AS AC 2D 7H
Tu mano: AH, AS, AC, 2D, 7H | Reglas: Trío | Modelo: Carta alta
→ Tienes Trío. Descarta las 2 sueltas: 2D, 7H. Conserva AH, AS, AC. Roba 2.
Indica las 2 carta(s) ROBADAS (ej. '2D 7H ...'): AD 6C
Mano final: AH, AS, AC, AD, 6C | Reglas: Póker | Modelo: Carta alta
Opciones: Pasar · Apostar · Igualar · Subir · Retirarse | P(ganar)≈99.2% · Breakeven igualar≈16.7%
Recomendación: **Apostar/Subir** (mano fuerte). Evita dar cartas gratis.
Jugada finalizada.
¿Nueva ronda? (S/N): N
Fin.
